# Project Planning Stage (Individual)

I will be working with log data from a Minecraft server
about the players and the sessions played on the Minecraft server.

In [ ]:
library(tidyverse)
library(lubridate)

players <- read_csv("players.csv")

sessions <- read_csv("sessions.csv")

glimpse(players)
glimpse(sessions)

In [ ]:
sessions_duration <- sessions |>
    mutate(
        start = dmy_hm(start_time), end = dmy_hm(end_time),
        duration_min = as.numeric(difftime(end, start, units = "mins")))

avg_session <- sessions_duration |>
    group_by(hashedEmail) |>
    summarize(
        avg_session_mins = mean(duration_min, na.rm = TRUE),
        n_sessions = n(),
        .groups = "drop")

data_analysis <- players |>
    left_join(avg_session, by = "hashedEmail") |>
    filter(!is.na(Age), !is.na(avg_session_mins))

glimpse(data_analysis)

# Data Description:

The "players" table has one row per player and it includes things like age in years and experience levels. The "sessions" table has one row per play session and records the start and end time of the session. Using the times given from the "sessions" table, I created the variable 'duration_min' that measures how long each session lasted in minutes. 

From there, I summarized this to get the average session minutes using the average session length for each player and the number of sessions they had. Then, I joined the tables together to form the variable 'data_analysis'. Players with a missing age or no recorded sessions were dropped as they had nothing to be calculated. Conclusively, most players in the 'data_analysis' were teenagers or young adults (with a few outliers). Their session lengths had variety with there being many short sessions and a few longer ones.

In [ ]:
data_analysis |>
    summarize(
        mean_age = mean(Age, na.rm = TRUE),
        mean_avg_session = mean(avg_session_mins, na.rm = TRUE),
        mean_n_sessions = mean(n_sessions, na.rm = TRUE),
        mean_played_hours = mean(played_hours, na.rm = TRUE))

# Questions:

*Broad Question*: "We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts."

*Specific question*: "Can a player's experience level and age help predict their average session length (minutes)?"

The response variable will be the average session length for each player that I solved for from the session log. The explanatory variable will be age and the experience level for each player. Therefore, this question fits the data well due it connecting background information to how people use the server.

In [ ]:
ggplot(data_analysis, aes(x = Age)) +
    geom_histogram(binwidth = 4) +
    labs(
        title = "Age Distribution of Players",
        x = "Age (years)",
        y = "Number of players")


In [ ]:
ggplot(data_analysis, aes(x = avg_session_mins)) +
    geom_histogram(binwidth = 8) +
    labs(
        title = "Average Session Length Distribution",
        x = "Average Session Length (mins)",
        y = "Number of players")

In [ ]:
ggplot(data_analysis, aes (x = Age, y = avg_session_mins)) +
    geom_point() +
    labs(
        title = "Age vs. Average Session Length",
        x = "Age",
        y = "Average Session Length (mins)")

# Exploratory Data Analysis and Visualization

The histogram of the players age distribution shows that most players are in their late teens and early twenties. Separately, the histogram showing the average session length is skewed to the right meaning that most players have playing sessions for less than 30 minutes, and there's a smaller group that has longer sessions.

The scatterplot of age vs. average session length doesn't really show any strong trends.

Overall, the plots show that experience could be more useful than age, but nothing can truly be confirmed.

# Methods and Plan

**I plan to use linear regression.**

*Why is this method appropriate?*

Because my outcome is numeric and I want to check whether age and experience relate to average session length.

*Which assumptions are required, if any, to apply the method selected?*

The assumptions required are independent observations, constant variance, and a roughly linear relationship.

*What are the potential limitations or weaknesses of the method selected?*

It could miss patterns that are non-linear and it won't be able to show causation, only association.

*How are you going to compare and select the model?*

I'll compare models using a basic residual check to see which version makes more sense.

*How are you going to process the data to apply the model?*

I'll clean and join the data, get rid of missing values, then split into training and test sets, then at this point, I'll fit the model.